In [1]:
import os

if "src" not in os.listdir():
    os.chdir("../")

In [13]:
import datasets
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM


MODEL_NAME = "unsloth/Qwen2.5-7B-Instruct"

In [22]:
df = pd.read_parquet("./data/interim/data_param_tunning/data_prompts.parquet")

In [23]:
df.head(3)

,Unnamed: 0,points,cat,prompt_dense_query,len_dense_query,prompt_dense_query_rerank,len_dense_query_rerank,prompt_dense_sparse_query,len_dense_sparse_query,prompt_dense_sparse_query_rerank,len_dense_sparse_query_rerank
0,0,12024168,train,<|im_start|>system\nТы юрист и эксперт по росс...,517,<|im_start|>system\nТы юрист и эксперт по росс...,488,<|im_start|>system\nТы юрист и эксперт по росс...,953,<|im_start|>system\nТы юрист и эксперт по росс...,968
1,1,854337,train,<|im_start|>system\nТы юрист и эксперт по росс...,1347,<|im_start|>system\nТы юрист и эксперт по росс...,1166,<|im_start|>system\nТы юрист и эксперт по росс...,893,<|im_start|>system\nТы юрист и эксперт по росс...,1023
2,2,4748322,train,<|im_start|>system\nТы юрист и эксперт по росс...,1299,<|im_start|>system\nТы юрист и эксперт по росс...,1448,<|im_start|>system\nТы юрист и эксперт по росс...,1596,<|im_start|>system\nТы юрист и эксперт по росс...,1627


In [24]:
col = "prompt_dense_query_rerank"

train = Dataset.from_dict({"text": df.loc[df["cat"] == "train", col].to_list()})
valid = Dataset.from_dict({"text": df.loc[df["cat"] == "valid", col].to_list()})

In [6]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.4: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.542 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth 2025.2.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

response_template = "<|im_start|>assistant\n"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [14]:
dataset = datasets.load_from_disk("./data/processed/100k_rows_qwen")
train = dataset["train"]
valid = dataset["valid"]

In [18]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train,
    eval_dataset=valid,
    dataset_text_field="prompt_context",
    max_seq_length=max_seq_length,
    data_collator=collator,
    dataset_num_proc=64,
    packing=False,  # Can make training 5x faster for short sequences.
    args=TrainingArguments(
        per_device_train_batch_size=16,
        gradient_accumulation_steps=2,
        warmup_steps=5,
        num_train_epochs=3,  # Set this for 1 full training run.
        # eval_strategy="epoch",
        eval_steps=50,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=100,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Map (num_proc=64):   0%|          | 0/81234 [00:00<?, ? examples/s]

Map (num_proc=64):   0%|          | 0/15000 [00:00<?, ? examples/s]

In [21]:
print(train["prompt_context"][0])

<|im_start|>system
Ты юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай, что тебе пишут, ответь на поставленные вопросы и дай консультацию. Ответ пиши только на РУССКОМ языке!<|im_end|>
<|im_start|>user
Кто может сделать форму P 13014?
Срочно.<|im_end|>
<|im_start|>system
Перед тем как дать консультацию изучи вопросы других людей и ответы на них других юристов:
Вопрос гражданина:
    Как заполнить ф 13014 для подтверждения сведений в ЕГРЮЛ по директору, заранее благодарю.
    Ответ юриста:
    Обратите внимание сюда, образцы заполнения
    Вопрос гражданина:
    Где можно взять копию формы 4?
    Ответ юриста:
    Смотрите ссылку:
    Вопрос гражданина:
    Где взять форму этой справки.
    Ответ юриста:
    Уважаемая Татьяна уточните вопрос, непонятно о какой справке идет речь.
    <|im_end|>
<|im_start|>assistant
Какие у вас сложности?<|im_end|>



In [3]:
len(
    """
<|im_start|>system
Ты юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай, что тебе пишут, ответь на поставленные вопросы и дай консультацию. Ответ пиши только на РУССКОМ языке!<|im_end|>
<|im_start|>user
Кто может сделать форму P 13014?
Срочно.<|im_end|>
<|im_start|>system
Перед тем как дать консультацию изучи вопросы других людей и ответы на них других юристов:
Вопрос гражданина:
    Как заполнить ф 13014 для подтверждения сведений в ЕГРЮЛ по директору, заранее благодарю.
    Ответ юриста:
    Обратите внимание сюда, образцы заполнения
    Вопрос гражданина:
    Где можно взять копию формы 4?
    Ответ юриста:
    Смотрите ссылку:
    Вопрос гражданина:
    Где взять форму этой справки.
    Ответ юриста:
    Уважаемая Татьяна уточните вопрос, непонятно о какой справке идет речь.
    <|im_end|>
<|im_start|>assistant
Какие у вас сложности?<|im_end|>
    """
)

888

In [25]:
print(train["text"][0])

<|im_start|>system
Ты юрист и эксперт по российскому праву, твоя задача помогать людям. Изучи вопрос пользователя, изучи переданные затем тебе вопросы других граждан и ответы, которые давали на них другие юристы. На основании этих данных ответь на вопрос гражданина. Ответ пиши только на РУССКОМ языке!<|im_end|>
<|im_start|>user
У меня просрочка  в 3 тысячи рублей в мфо, нет денег заплатить так как сижу временно без работы, что будет если не заплачу. Брал займ онлайн без посещения офиса.<|im_end|>
<|im_start|>system
Прежде чем дать ответ изучи вопросы других граждан и ответы на них юристов:

Вопрос гражданина:
У меня просрочка  по платежам в мфо, что делать денег нет потерял работу звонки постоянно. Спасибо.

Ответ юриста:
В зависимости от того, чего хотите добиться.

Вопрос гражданина:
Брала займ МФО через интернет  месяц оплатила. Сейчас возможности оплатить нет просрочка три месяца. Подскажите пожалуйста чем мне это грозит. Они мне пока не звонят.

Ответ юриста:
МФО вправе обратиться

In [19]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 81,234 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 2
\        /    Total batch size = 32 | Total steps = 7,617
 "-____-"     Number of trainable parameters = 40,370,176


Step,Training Loss


KeyboardInterrupt: 